In [1]:
import os
import glob
import pandas as pd

In [17]:
#old = pd.read_csv("../../src/processing_area_nov10.csv")
#new = pd.read_csv("../../src/lac_africa_2022.csv")
new = pd.read_csv("process_area_2022.csv")
print(len(new))



1292830


In [19]:
df = new.filter(['X', 'Y', 'Y_tile', 'X_tile', 'sovereignt'])

In [20]:
df.to_csv("process_area_2022.csv", index = False)

In [18]:
new.head(5)

,X,Y,Y_tile,X_tile,country,sovereignt,n,distance,feature_x,feature_y,nearest_x,nearest_y
0,-116.9722,37.4722,1624,0,United States of America,United States of America,1.0,0.0,-116.9722,37.4722,-116.9722,37.4722
1,-116.9167,37.4722,1624,1,United States of America,United States of America,1.0,0.0,-116.9167,37.4722,-116.9167,37.4722
2,-116.8611,37.4722,1624,2,United States of America,United States of America,1.0,0.0,-116.8611,37.4722,-116.8611,37.4722
3,-116.5833,37.4722,1624,3,United States of America,United States of America,1.0,0.0,-116.5833,37.4722,-116.5833,37.4722
4,-116.5278,37.4722,1624,4,United States of America,United States of America,1.0,0.0,-116.5278,37.4722,-116.5278,37.4722


In [7]:
import numpy as np
new['Y'] = np.around(new['Y'], 4)
old['Y'] = np.around(old['Y'], 4)
new['X'] = np.around(new['X'], 4)
old['X'] = np.around(old['X'], 4)

In [8]:
old_y = np.unique(old['Y'])
new_y = np.unique(new['Y'])

old_x = np.unique(old['X'])
new_x = np.unique(new['X'])

to_remove_y = [x for x in new_y if x not in old_y]
to_remove_x = [x for x in new_x if x not in old_x]
print(len(to_remove_x))
print(len(to_remove_y))

for i in to_remove_y:
    new = new[new.Y != i]
    
for i in to_remove_x:
    new = new[new.X != i]

165
49


In [9]:
new.head(5)

,X,Y,VALUE,Unnamed: 3
0,-116.9722,37.4722,1.0,NaN
1,-116.9167,37.4722,1.0,NaN
2,-116.8611,37.4722,1.0,NaN
7,-116.5833,37.4722,1.0,NaN
8,-116.5278,37.4722,1.0,NaN


In [10]:
new = pd.merge(new, old, on = ['X', 'Y'], how = 'left')




In [11]:
new.to_csv("temp.csv")



In [12]:
len(new)

1292817

In [12]:
def id_tile_number(new, old, idx):
    if np.isnan(new['Y_tile'][idx]):
        new_y = new['Y'][idx]
        new_x = new['X'][idx]
        y_tile = np.unique(old[old['Y'] == new_y]['Y_tile'])
        x_tile = np.unique(old[old['X'] == new_x]['X_tile'])
        assert len(y_tile) == 1
        assert len(x_tile) == 1
        new['X_tile'][idx] = x_tile[0]
        new['Y_tile'][idx] = y_tile[0]


In [13]:
import warnings 
warnings.simplefilter(action='ignore') 
from tqdm.notebook import tnrange
for row in tnrange(len(new)):
    id_tile_number(new, old, row)

  0%|          | 0/1292817 [00:00<?, ?it/s]

In [14]:
new.head(5)

,X,Y,VALUE,Unnamed: 3,Y_tile,X_tile,country
0,-116.9722,37.4722,1.0,NaN,1624.0,0.0,United States of America
1,-116.9167,37.4722,1.0,NaN,1624.0,1.0,United States of America
2,-116.8611,37.4722,1.0,NaN,1624.0,2.0,United States of America
3,-116.5833,37.4722,1.0,NaN,1624.0,3.0,United States of America
4,-116.5278,37.4722,1.0,NaN,1624.0,4.0,United States of America


In [15]:
new = new.filter(['X', 'Y', 'Y_tile', 'X_tile', 'country'])

In [16]:
new.to_csv("../../src/temp2.csv", index = False)




In [ ]:
base_folder = "../project-monitoring/tof/"
from shutil import copyfile, copytree

for folder_y in os.listdir(base_folder):
    if "." not in folder_y:
        for folder_x in os.listdir(base_folder + folder_y):
            if "." not in folder_x:
                for file in os.listdir(base_folder + folder_y + "/" + folder_x):
                    if file[-4:] == ".tif":
                        source_file = file
                        source = old[old['X_tile'].astype(str) == folder_y]
                        source = source[source['Y_tile'].astype(str) == folder_x]
                        source = source.reset_index(drop = True)
                        source_y = source['Y'][0]
                        source_x = source['X'][0]
                        dest = new[np.isclose(new['Y'], source_y)]
                        dest = dest[np.isclose(dest['X'], source_x)]
                        dest = dest.reset_index(drop = True)
                        dest_x = dest['X_tile'][0]
                        dest_y = dest['Y_tile'][0]
                        #print(folder_y, dest_x)
                        #print(folder_x, dest_y)
                        
                        new_dir = "../project-monitoring/tiles/" + str(dest_x) + "/" + str(dest_y) + "/"
                        new_file = f'{new_dir}{dest_x}X{dest_y}Y_v1.tif'
                        old_file = f'{base_folder}{folder_y}/{folder_x}/{file}'
                        
                        
                        if not os.path.exists(os.path.realpath(new_dir)):
                            os.makedirs(os.path.realpath(new_dir))
                        copyfile(old_file, new_file)

# Create temp copy of individual country for reprocessing

In [ ]:
df = new[new['country'] == 'Rwanda']
df.head(5)

In [ ]:
from shutil import copyfile, copytree

for index, row in df.iterrows():
    x = str(int(row['X_tile']))
    y = str(int(row['Y_tile']))
    old_path = f'../../project-monitoring/tof-output/{str(x)}/{str(y)}/'
    new_path = f'../../project-monitoring/rwanda/{str(x)}/{str(y)}/'
    if os.path.isdir(old_path):
        copytree(old_path, new_path)
